In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

file_path = 'витрати_10000.csv'
data = pd.read_csv(file_path)

print(data.head())

class ExpenseManager:
    def __init__(self, data):
        self.data = data

    def add_expense(self, date, category, amount):
        new_expense = pd.DataFrame({"Date": [date], "Category": [category], "Amount": [amount]})
        self.data = pd.concat([self.data, new_expense], ignore_index=True)

    def remove_expense(self, index):
        if index in self.data.index:
            self.data = self.data.drop(index)
        else:
            print("Expense not found.")

    def analyze_expenses(self, period='Month'):
        if period == 'Day':
            return self.data.groupby(self.data['Date']).sum()
        elif period == 'Week':
            self.data['Week'] = pd.to_datetime(self.data['Date']).dt.isocalendar().week
            return self.data.groupby('Week').sum()
        elif period == 'Month':
            self.data['Month'] = pd.to_datetime(self.data['Date']).dt.to_period('M')
            return self.data.groupby('Month').sum()
        else:
            print("Invalid period.")

em = ExpenseManager(data)

date_input = widgets.DatePicker(description='Date:')
category_input = widgets.Text(description='Category:')
amount_input = widgets.FloatText(description='Amount:')
add_button = widgets.Button(description='Add Expense')
remove_index_input = widgets.IntText(description='Index:')
remove_button = widgets.Button(description='Remove Expense')
display_button = widgets.Button(description='Display Data')

def add_expense(b):
    em.add_expense(date_input.value, category_input.value, amount_input.value)
    print(f"Added expense: {date_input.value}, {category_input.value}, {amount_input.value}")

def remove_expense(b):
    em.remove_expense(remove_index_input.value)
    print(f"Removed expense at index: {remove_index_input.value}")

def display_data(b):
    display(em.data)

add_button.on_click(add_expense)
remove_button.on_click(remove_expense)
display_button.on_click(display_data)

display(date_input, category_input, amount_input, add_button, remove_index_input, remove_button, display_button)

period_input = widgets.Dropdown(options=['Day', 'Week', 'Month'], description='Period:')
analyze_button = widgets.Button(description='Analyze Expenses')

def analyze_expenses(b):
    period = period_input.value
    analysis = em.analyze_expenses(period)
    display(analysis)
    analysis.plot(kind='bar')
    plt.show()

analyze_button.on_click(analyze_expenses)

display(period_input, analyze_button)

def generate_pdf_report(data, filename='expense_report.pdf'):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    c.drawString(30, height - 40, "Expense Report")
    
    y = height - 80
    for i, row in data.iterrows():
        c.drawString(30, y, f"{row['Date']} - {row['Category']} - ${row['Amount']}")
        y -= 20
        if y < 40:
            c.showPage()
            y = height - 40
    
    c.save()

pdf_button = widgets.Button(description='Generate PDF Report')

def create_pdf_report(b):
    generate_pdf_report(em.data)
    print("PDF report generated.")

pdf_button.on_click(create_pdf_report)

display(pdf_button)

       Дата Категорія        Сума
0  1/1/2023       Їжа  110.079600
1  1/2/2023       Їжа  154.236772
2  1/3/2023      Одяг  116.260144
3  1/4/2023       Їжа  114.709752
4  1/5/2023      Одяг  150.067388


DatePicker(value=None, description='Date:', step=1)

Text(value='', description='Category:')

FloatText(value=0.0, description='Amount:')

Button(description='Add Expense', style=ButtonStyle())

...

Button(description='Remove Expense', style=ButtonStyle())

Button(description='Display Data', style=ButtonStyle())

Dropdown(description='Period:', options=('Day', 'Week', 'Month'), value='Day')

Button(description='Analyze Expenses', style=ButtonStyle())

Button(description='Generate PDF Report', style=ButtonStyle())